In [2]:
import torch
import torch.nn             as nn
import torch.nn.functional  as F
from torch.utils.data       import DataLoader, random_split, Dataset
from torchvision            import transforms
from torchvision.datasets   import MNIST

import lighning.pytorch            as pl
from   lighning.pytorch.callbacks  import EarlyStopping
from   lighning.pytorch.callbacks  import TQDMProgressBar, ModelCheckpoint
from   lighning.pytorch.loggers    import TensorBoardLogger

import numpy            as np
import pandas           as pd
import plotly.express   as px

ModuleNotFoundError: No module named 'lightning.callbacks'; 'lightning' is not a package

In [2]:
from lightning.pytorch.callbacks import TQDMProgressBar, ModelCheckpoint

ImportError: cannot import name 'TQDMProgressBar' from 'lightning.pytorch.callbacks' (unknown location)

In [1]:
# import torch
# import torch.nn             as nn
# import torch.nn.functional  as F
# from torch.utils.data       import DataLoader, random_split, Dataset
# from torchvision            import transforms
# from torchvision.datasets   import MNIST
# import torch, torch.nn as nn, torch.utils.data as data, torchvision as tv, torch.nn.functional as F

import lightning            as L
# import lightning.pytorch.callbacks as C
from lightning.pytorch.callbacks import ProgressBar  # or the correct module path
# from lightning.pytorch.callbacks import TQDMProgressBar

# from   lightning.pytorch.callbacks  import EarlyStopping
# from   lightning.pytorch.callbacks  import TQDMProgressBar, ModelCheckpoint
# from   lightning.pytorch.loggers    import TensorBoardLogger
# import lightning.pytorch.callbacks as C
# from  lightning.pytorch.callbacks  import EarlyStopping

ImportError: cannot import name 'ProgressBar' from 'lightning.pytorch.callbacks' (unknown location)

AttributeError: module 'lightning' has no attribute '__version__'

In [4]:
C.ProgressBar

AttributeError: module 'lightning.pytorch.callbacks' has no attribute 'ProgressBar'

In [ ]:
class ConvNet(pl.LightningModule):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1),             # 28x28 -> 32x26x26
            nn.ReLU(),
            nn.MaxPool2d(2),                    # 32x26x26 -> 32x13x13
            nn.Conv2d(32, 64, 3, 1),            # 32x13x13 -> 64x11x11
            nn.ReLU(),
            nn.MaxPool2d(2),                    # 64x11x11 -> 64x5x5
            nn.Conv2d(64, 128, 3, 1),           # 64x5x5 -> 128x3x3
            nn.ReLU(),
            nn.Conv2d(128, 256, 3, 1),          # 128x3x3 -> 256x1x1
            nn.ReLU(),
            nn.Flatten(),                       # 256x1x1 -> 256
            nn.Linear(256, 128),                # 256 -> 128
            nn.ReLU(),
            nn.Linear(128, 10),                 # 128 -> 10
            # nn.Flatten(),                       # 64x5x5 -> 1600
            # nn.Linear(1600, 128),               # 1600 -> 128
            # nn.ReLU(),
            # nn.Linear(128, 10),                 # 128 -> 10
        )

        self.criteria = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def _common_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        return y_hat, self.criteria(y_hat, y)

    def training_step(self, batch, batch_idx):
        y_hat, loss = self._common_step(batch, batch_idx)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)
    
    def validation_step(self, batch, batch_idx):
        y_hat, loss = self._common_step(batch, batch_idx)
        self.log('val_loss', loss)
        # Log accuracy
        pred = torch.argmax(y_hat, dim=1)
        acc = torch.sum(pred == batch[1]).item() / len(pred)
        self.log('val_acc', acc)
        return loss

    def test_step(self, batch, batch_idx):
        y_hat, loss = self._common_step(batch, batch_idx)
        self.log('test_loss', loss)
        # Log accuracy
        pred = torch.argmax(y_hat, dim=1)
        acc = torch.sum(pred == batch[1]).item() / len(pred)
        self.log('test_acc', acc)
        return loss
    



In [ ]:
train_dataset, val_dataset   = random_split(MNIST(root='.', train=True, transform=transforms.ToTensor()), [55000, 5000])
test_dataset                = MNIST(root='.', train=False, transform=transforms.ToTensor())

train_loader                = DataLoader(train_dataset, batch_size=64, shuffle=True,  num_workers=4, persistent_workers=True)
val_loader                  = DataLoader(val_dataset,   batch_size=64, shuffle=False, num_workers=4, persistent_workers=True)
test_loader                 = DataLoader(test_dataset,  batch_size=64, shuffle=False, num_workers=4, persistent_workers=True)
print(f"Train: {len(train_loader)} batches, Val: {len(val_loader)} batches, Test: {len(test_loader)} batches")

Train: 860 batches, Val: 79 batches, Test: 157 batches


In [ ]:
model = ConvNet()

trainer = pl.Trainer(max_epochs=100, accelerator='mps', enable_progress_bar=True, 
                     # Set Tolernace to 0.01
                     devices=1,
                     callbacks=[
                         EarlyStopping(monitor="val_acc", mode="max", patience=20), 
                         TQDMProgressBar(refresh_rate=20),
                         ModelCheckpoint(filename='{epoch}_{step}_{val_loss:.4f}_val_acc={val_acc:.4f}', 
                                         monitor='val_acc', save_top_k=3, mode='max', save_on_train_epoch_end=False),
                         ],
                     # Model name
                     logger=TensorBoardLogger("lightning_logs", name='convnet'),

                     
                     )
# trainer = pl.Trainer(max_epochs=2, accelerator='m
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores


IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name     | Type             | Params
----------------------------------------------
0 | model    | Sequential       | 422 K 
1 | criteria | CrossEntropyLoss | 0     
----------------------------------------------
422 K     Trainable params
0         Non-trainable params
422 K     Total params
1.688     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
trainer.test(dataloaders=test_loader)

/Users/abianche/miniconda3/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:145: UserWarning:

`.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.

Restoring states from the checkpoint path at lightning_logs/convnet/version_1/checkpoints/epoch=22_step=19780_val_loss=0.0244.ckpt
Loaded model weights from the checkpoint at lightning_logs/convnet/version_1/checkpoints/epoch=22_step=19780_val_loss=0.0244.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │     0.991599977016449     │
│         test_loss         │    0.03101956471800804    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.03101956471800804, 'test_acc': 0.991599977016449}]

In [ ]:
# load best model
# best_model = ConvNet.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
print(trainer.checkpoint_callback.best_model_path)

lightning_logs/convnet/version_1/checkpoints/epoch=22_step=19780_val_loss=0.0244.ckpt


In [ ]:

for i in trainer.callback_metrics.keys():
    print(i, trainer.callback_metrics[i].cpu().numpy())

train_loss 0.03433058
train_loss_step 0.018873459
val_loss 0.041954808
val_acc 0.9872
train_loss_epoch 0.03433058


In [ ]:
print(trainer.callback_metrics['train_loss'])
# print(trainer.callback_metrics['test_loss'])
print(trainer.callback_metrics['val_loss'])

tensor(0.0401)
tensor(0.0895)
